In [2]:
import pandas as pd

# Data Processing

load data, rename columns, and make a dictionary so we can reference back to the verbose names given in the raw survey data

In [3]:
# Task 1 - 2
df = pd.read_csv('./data/survey_raw.csv')

names = ['timestamp', 'id', 'freq_browser', 'freq_phone',
         'length_use', 'freq_use', 'ability_find_tasks',
         'promptly_find_tasks', 'text_search_preference',
         'notification_preference', 'message_preference',
         'understood', 'browsers', 'work_aids', 'fora',
         'other_markets', 'other_markets_length_use',
         'authority_comfort', 'collective_individual_scale',
         'planning_scale', 'timeliness_scale', 'emotion_scale',
         'success_rating', 'leadership_rating', 'leadership_preference',
         'gender_preference', 'multitasking', 'short_term_plans',
         'linear_work_style', 'plan_change_aversion', 'plan_change_adaptable',
         'lateness_aversion', 'blank', 'spare_time_alone',
         'commitment_others_over_self', 'close_over_casual_friends',
         'internet_friends', 'support_from_friends', 'pandemic_effect_magnitude',
         'chosen_task_completion_confidence', 'complex_task_completion_confidence',
         'neutral', 'task_satisfaction', 'likelihood_continued_use',
         'task_motivation', 'plugin_preference', 'current_country',
         'home_country', 'only_one_country', 'education', 'gender', 'age']

aliases = dict(zip(names, df.columns))

df.columns = names

In [4]:
df

,timestamp,id,freq_browser,freq_phone,length_use,freq_use,ability_find_tasks,promptly_find_tasks,text_search_preference,notification_preference,...,task_satisfaction,likelihood_continued_use,task_motivation,plugin_preference,current_country,home_country,only_one_country,education,gender,age
0,1/13/2022 16:04:26,2aef4422f3276e44b1eea0c2a89374,Always,Rarely,More than 3 years,Everyday,Neutral,Neutral,Agree,Agree,...,Neutral,NaN,NaN,NaN,Belarus,Belarus,Yes,Associate degree,Male,35-44 years old
1,1/13/2022 21:43:07,af2a7da96237f5baa54f6e67c1c0f1,Always,Always,Less than a month,Everyday,Strongly Agree,Agree,Agree,Strongly Agree,...,Agree,NaN,NaN,NaN,United States,Mexico,Yes,Bachelor’s degree,Male,18-24 years old
2,1/14/2022 3:59:47,1cb48555a624b4c6fbe5e811271eb,Always,Sometimes,Between 1 and 2 years,From three to four days a week,Agree,Neutral,Agree,Strongly Agree,...,Agree,NaN,NaN,NaN,Russia,Russia,Yes,"Some high school, no diploma",Male,25-34 years old
3,1/21/2022 11:27:52,8d87dd60d895e59dd459a6593144461,Always,Sometimes,Between 6 months and 1 year,Everyday,Neutral,Neutral,Neutral,Disagree,...,Strongly Agree,NaN,NaN,NaN,Philippines,Philippines,Yes,Bachelor’s degree,Male,25-34 years old
4,1/21/2022 16:18:18,e2f37f95c0b82dc037a56f768d591b1e,Always,Always,Between 1 and 3 months,From three to four days a week,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,...,Agree,4.0,5.0,4.0,United States,United States,Yes,Bachelor’s degree,Male,18-24 years old
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,2/12/2022 11:53:35,d2e520c281416d65bb31f09755cdae20,Always,Always,Between 3 and 6 months,Everyday,Agree,Agree,Strongly Agree,Agree,...,Slightly Agree,3.0,4.0,2.0,Jordan,Jordan,Yes,"High school graduate, diploma or the equivalen...",Female,25-34 years old
85,2/12/2022 13:45:23,830959a841ecfe33f83e09e7f114476,Always,Always,Between 6 months and 1 year,Everyday,Strongly disagree,Strongly disagree,Strongly disagree,Strongly disagree,...,Strongly Agree,4.0,5.0,5.0,Venezuela,Venezuela,Yes,"High school graduate, diploma or the equivalen...",Female,35-44 years old
86,2/12/2022 21:13:51,3741da8e9dd5e15f64a570dbb218a7,Always,Often,More than 2 and 3 years,Everyday,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,...,Strongly Agree,5.0,5.0,5.0,India,India,Yes,Master’s degree,Male,35-44 years old
87,2/13/2022 11:12:42,69c0d3e67052b28759e150bca54f5f0,Always,Rarely,Between 3 and 6 months,From three to four days a week,Agree,Agree,Agree,Agree,...,Neutral,3.0,5.0,4.0,United States,United States,No,Master’s degree,Female,25-34 years old


In [ ]:
# Task 3

In [6]:
# Task 4
# Create script that can group workers based on whether they are from the global south or global north

# Approach 1 - add a column indicating whether the worker is from Global North or South
# Approach 2?

# Inspect what countries are the workers from
df['home_country'].unique()

array(['Belarus', 'Mexico', 'Russia', 'Philippines', 'United States',
       'United Kingdom', 'Ghana', 'Jordan', 'Dominican Republic',
       'Nigeria', 'Pakistan', 'Brazil', 'Costa Rica', 'Vietnam',
       'Venezuela', 'Canada', 'Kenya', 'Egypt', 'Morocco', 'India'],
      dtype=object)

In [23]:
# Get a list of countries for Global North & South
# data source: https://github.com/wikimedia-research/canonical-data/blob/master/countries.csv

# NOTE: I think global south and north has its own definition, rather than mono and poly
# but not sure if this is what the Prof. wants, can double check :)

countries = pd.read_csv('./data/countries.csv')
countries.index = countries.index + 1
countries = countries.iloc[:-1 , :][["name","economic_region"]]
countries

,name,economic_region
1,Afghanistan,Global South
2,Åland Islands,Global North
3,Albania,Global North
4,Algeria,Global South
5,American Samoa,Global South
...,...,...
246,Wallis and Futuna,Global South
247,Western Sahara,Global South
248,Yemen,Global South
249,Zambia,Global South


In [22]:
global_south = list(countries[countries['economic_region'] == 'Global South']['name'])
global_south

['Afghanistan',
 'Algeria',
 'American Samoa',
 'Angola',
 'Anguilla',
 'Antarctica',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Botswana',
 'Brazil',
 'British Indian Ocean Territory',
 'British Virgin Islands',
 'Brunei',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Cape Verde',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Christmas Island',
 'Cocos Islands',
 'Colombia',
 'Comoros',
 'Cook Islands',
 'Costa Rica',
 "Côte d'Ivoire",
 'Cuba',
 'Democratic Republic of the Congo',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Ethiopia',
 'Falkland Islands',
 'Federated States of Micronesia',
 'Fiji',
 'French Guiana',
 'French Polynesia',
 'Gabon',
 'Gambia',
 'Georgia',
 'Ghana',
 'Grenada',
 'Guadeloupe',
 'Guam',
 'Guatemal

In [21]:
global_north = list(countries[countries['economic_region'] == 'Global North']['name'])
global_north

['Åland Islands',
 'Albania',
 'Andorra',
 'Australia',
 'Austria',
 'Belgium',
 'Bermuda',
 'Bosnia and Herzegovina',
 'Bouvet Island',
 'Bulgaria',
 'Canada',
 'Caribbean Netherlands',
 'Croatia',
 'Curaçao',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Estonia',
 'Faroe Islands',
 'Finland',
 'France',
 'French Southern and Antarctic Lands',
 'Germany',
 'Gibraltar',
 'Greece',
 'Greenland',
 'Guernsey',
 'Hong Kong',
 'Hungary',
 'Iceland',
 'Ireland',
 'Isle of Man',
 'Israel',
 'Italy',
 'Japan',
 'Jersey',
 'Latvia',
 'Liechtenstein',
 'Lithuania',
 'Luxembourg',
 'Macao',
 'Macedonia',
 'Malta',
 'Moldova',
 'Monaco',
 'Montenegro',
 'Netherlands',
 'New Zealand',
 'Norway',
 'Poland',
 'Portugal',
 'Romania',
 'Russia',
 'Saint Pierre and Miquelon',
 'San Marino',
 'Serbia',
 'Singapore',
 'Slovakia',
 'Slovenia',
 'South Korea',
 'Spain',
 'Svalbard and Jan Mayen',
 'Sweden',
 'Switzerland',
 'Taiwan',
 'Turkey',
 'United Kingdom',
 'United States',
 'Vatican City']

In [28]:
# Add a column indicating whether the worker is from Global North or South
def replace_region(row):
    if row['home_country'] in global_south:
        return 'global_south'
    
    if row['home_country'] in global_north:
        return 'global_north'
    else: 
        print ("Error!")
        
df['region'] = df.apply(lambda row: replace_region(row), axis=1)
df[['id', 'home_country','region']]

# fron here we can slice the data based on region

,id,home_country,region
0,2aef4422f3276e44b1eea0c2a89374,Belarus,global_south
1,af2a7da96237f5baa54f6e67c1c0f1,Mexico,global_south
2,1cb48555a624b4c6fbe5e811271eb,Russia,global_north
3,8d87dd60d895e59dd459a6593144461,Philippines,global_south
4,e2f37f95c0b82dc037a56f768d591b1e,United States,global_north
...,...,...,...
84,d2e520c281416d65bb31f09755cdae20,Jordan,global_south
85,830959a841ecfe33f83e09e7f114476,Venezuela,global_south
86,3741da8e9dd5e15f64a570dbb218a7,India,global_south
87,69c0d3e67052b28759e150bca54f5f0,United States,global_north
